In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import TensorDataset,random_split
import torch.nn.functional as F
import torchvision
import PIL
import matplotlib.pyplot as plt
import pandas as pd
import zipfile
from tqdm.auto import tqdm
import math
import torch.nn as nn

In [ ]:
device = torch.device('cuda')
dtype = torch.float32

In [ ]:
Full_dataset = torch.load('/content/drive/MyDrive/Colab Notebooks/Aoa1_dataset.pt')
#%%
# n_train = int(len(Full_dataset)*0.7)+1
# n_val = int(len(Full_dataset)*0.2)
# n_test = int(len(Full_dataset)*0.1)
#%%
Train_set,Val_set = random_split(Full_dataset,[2293760,573440])
# %%
batch_size = 4096
#%%
Train_loader = torch.utils.data.DataLoader(
       dataset=Train_set,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True)

Val_loader = torch.utils.data.DataLoader(
       dataset=Val_set,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True)

In [ ]:
import torch.nn.functional as F
class Classification(nn.Module):
    def __init__(self, n_channels=16,channel1=64,channel2=128,channel3=256,channel4=512, channel5=1024, channel6=2048, n_classes=14):
        super().__init__()
        #####################Encoder layers##############################
        self.encoder_1 = nn.Sequential(
            nn.Linear(n_channels, channel1),
            nn.BatchNorm1d(channel1),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )

        self.encoder_2 = nn.Sequential(
            nn.Linear(channel1, channel1),
            nn.BatchNorm1d(channel1),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )

        self.encoder_3 = nn.Sequential(
            nn.Linear(channel1, channel2),
            nn.BatchNorm1d(channel2),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )
        self.encoder_4 = nn.Sequential(
            nn.Linear(channel2, channel3),
            nn.BatchNorm1d(channel3),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )
        self.encoder_5 = nn.Sequential(
            nn.Linear(channel3, channel4),
            nn.BatchNorm1d(channel4),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )
        self.encoder_6 = nn.Sequential(
            nn.Linear(channel4, channel5),
            nn.BatchNorm1d(channel5),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )
        self.encoder_7 = nn.Sequential(
            nn.Linear(channel5, channel6),
            nn.BatchNorm1d(channel6),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )
        self.encoder_8 = nn.Sequential(
            nn.Linear(channel6, channel3),
            nn.BatchNorm1d(channel3),
            nn.ReLU(inplace = True),
            nn.Dropout(0.2)
        )
        self.regression1 = nn.Sequential(
            nn.Linear(channel3, 1),
        )
    def forward(self,x):

        x = self.encoder_1(x)

        x = self.encoder_2(x)

        x = self.encoder_3(x)

        x = self.encoder_4(x)

        x = self.encoder_5(x)

        x = self.encoder_6(x)

        x = self.encoder_7(x)

        x = self.encoder_8(x)

        reg1 = self.regression1(x)

        return reg1

In [ ]:
#%%
model = Classification()
weight_decay = 1e-6
optimizer = optim.Adam(model.parameters(), lr=0.00001,weight_decay=weight_decay)
model = model.to(device=device)  # move the model parameters to CPU/GPU
epochs = 30

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab Notebooks/e1h20_aoa1_regression.pth"))

<All keys matched successfully>

In [ ]:

#%%
for e in range(epochs):
    for t, (x, y) in enumerate(tqdm(Train_loader)):
        x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y = y.to(device=device, dtype=dtype)
        model.train()  # put model to training mode
        optimizer.zero_grad()

        aoa1 = model(x)
        aoa1 = torch.squeeze(aoa1)
        loss_regression = F.mse_loss(aoa1, y[:,0])
        loss_regression = torch.sqrt(loss_regression)

        # Zero out all of the gradients for the variables which the optimizer
        # will update.

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        # loss_regression.backward()
        # else:

        # optimizer.zero_grad()
        loss_regression.backward()
        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()
        # print(f'prediction {doa1} \n truth {y[0]}')
        # print(f'classification {classification[0]+1}')
    if e%2 == 0:
        print('Epoch %d, Regression loss = %.4f' % (e, loss_regression.item()))
            # print()
            # print(scores[0,:])
            # print(y[0,:])
    with torch.no_grad():
        model.eval()
        for idx,(x,y) in enumerate(tqdm(Val_loader)):
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=dtype)
                reg1 = model(x)
                loss_regression = F.mse_loss(torch.squeeze(reg1), y[:,0])
                loss_regression = torch.sqrt(loss_regression)
        if e%2 == 0:
            print('Epoch %d, Classification loss = %.4f' % (e, loss_regression))
            print(f'Epoch {e}, Truth ={y[0,:]}')
            print(f'Epoch {e}, sources ={reg1[0]}')

# %%


  0%|          | 0/560 [00:00<?, ?it/s]

Epoch 0, Regression loss = 8.4841


  0%|          | 0/140 [00:00<?, ?it/s]

Epoch 0, Classification loss = 6.9632
Epoch 0, Truth =tensor([0., 0., 0., 0., 0.], device='cuda:0')
Epoch 0, sources =tensor([-0.1755], device='cuda:0')


  0%|          | 0/560 [00:00<?, ?it/s]

...

Epoch 28, Regression loss = 8.5627


  0%|          | 0/140 [00:00<?, ?it/s]

Epoch 28, Classification loss = 6.9563
Epoch 28, Truth =tensor([41.,  0.,  0.,  0.,  0.], device='cuda:0')
Epoch 28, sources =tensor([41.1631], device='cuda:0')


  0%|          | 0/560 [00:00<?, ?it/s]

  0%|          | 0/140 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/Colab Notebooks/e1h50_aoa1_regression.pth')